### Matrix Multiplication
<br/>
<br/>
Matrix-matrix multiplication (MMM) is an important practical operation from which many applications demand high performance.
<br/>
Indeed, applying the mathematical definition* of MMM for two square matrix of dimension n gives a complexity order of calcul in $\mathcal{O}(n^3)$.
<br/>
<br/>
_______________________________*Mathematical definition_______________________________________
<br/>
<br/>
for A,B two matrix respectively of size $n\times m$ and $m\times p$, the components of the resulting matrix C of size $n\times p$ are given by : 
<br/>
$C_{i,j}=\Sigma_{k=1}^m A_{i,k}B_{k,i}$

In [1]:
#a useful function to create empty matrix as we don't use numpy
def zeros_matrix(rows, cols):
    """
    Creates a matrix filled with zeros.
        :param rows: the number of rows the matrix should have
        :param cols: the number of columns the matrix should have
 
        :return: list of lists that form the matrix
    """
    M = []
    while len(M) < rows:
        M.append([])
        while len(M[-1]) < cols:
            M[-1].append(0.0)
 
    return M

def dummy_matmul(A, B):
    """
    Returns the product of the matrix A * B according to the mathematical definition of MMM
        :param A: The first matrix - orders matter.
        :param B: The second matrix
 
        :return: The product of the two matrices
    """
    # Section 1: Ensure A & B dimensions are correct for multiplication
    rowsA = len(A)
    colsA = len(A[0])
    rowsB = len(B)
    colsB = len(B[0])
    if colsA != rowsB:
        raise ArithmeticError(
            'Number of A columns must equal number of B rows.')
    # Section 2: Store matrix multiplication in a new matrix
    C = zeros_matrix(rowsA, colsB)
    for i in range(rowsA):
        for j in range(colsB):
            total = 0
            for k in range(colsA):
                total += A[i][k] * B[k][j]
            C[i][j] = total
 
    return C

The problem is obvious here as there are three "for" interlinked. For bigger shapes it is problematic to calcul many differents MMM in a reasonable amount of time (as in a DL algorithm for instance).
<br/>
<br/>
Unfortunately, there is no magical solution to this problem. The better complexity that we can get currently for square matrix is $\mathcal{O}(2^{2.3728639})$ with the Coppersmith-Winograd algorithm. It is based on the Strasser algorithm (itself based on the divide and conquer algorithm). Below i implemented a simple version of Strasser algorithm for square matrices.

In [53]:
import numpy as np 
  
def split(matrix): 
    """ 
    Splits a given matrix into quarters. 
    Input: nxn matrix 
    Output: tuple containing 4 n/2 x n/2 matrices corresponding to a, b, c, d 
    """
    row, col = matrix.shape 
    row2, col2 = row//2, col//2
    return matrix[:row2,:col2], matrix[:row2,col2:], matrix[row2:,:col2], matrix[row2:,col2:] 
  
def strassen(x, y): 
    """ 
    Computes matrix product by divide and conquer approach, recursively. 
    Input: nxn matrices x and y 
    Output: nxn matrix, product of x and y 
    """
    # Base case when size of matrices is 1x1 
    if len(x) == 1: 
        return x * y 
  
    # Splitting the matrices into quadrants. This will be done recursively 
    # untill the base case is reached. 
    a, b, c, d = split(x) 
    e, f, g, h = split(y) 
  
    # Computing the 7 products, recursively (p1, p2...p7) 
    p1 = strassen(a, f - h)   
    p2 = strassen(a + b, h)         
    p3 = strassen(c + d, e)         
    p4 = strassen(d, g - e)         
    p5 = strassen(a + d, e + h)         
    p6 = strassen(b - d, g + h)   
    p7 = strassen(a - c, e + f)   
  
    # Computing the values of the 4 quadrants of the final matrix c 
    c11 = p5 + p4 - p2 + p6   
    c12 = p1 + p2            
    c21 = p3 + p4             
    c22 = p1 + p5 - p3 - p7   
  
    # Combining the 4 quadrants into a single matrix by stacking horizontally and vertically. 
    c = np.vstack((np.hstack((c11, c12)), np.hstack((c21, c22))))  
  
    return c 
x=np.random.randint(10,size=(4,4))
y=np.random.randint(10,size=(4,4))
print("strassen : {} \nnumpy.dot : {}".format(strassen(x,y), np.dot(x,y)))

strassen : [[ 77  86 176 115]
 [ 12  20  57  51]
 [ 50  48 113  85]
 [117  82 165 101]] 
numpy.dot : [[ 77  86 176 115]
 [ 12  20  57  51]
 [ 50  48 113  85]
 [117  82 165 101]]


### Useful links :
<br/>

- https://en.wikipedia.org/wiki/Matrix_multiplication_algorithm
- https://www.geeksforgeeks.org/strassens-matrix-multiplication/
- Very interesting paper on the computational view of matrix multiplication : https://arxiv.org/pdf/1702.02017.pdf
